In [1]:
# to capture game screen
from mss import mss
# to provide keystroke inputs to the game
import pydirectinput
import numpy as np
# to perform OCR on game screen
import time
# to create custom gym environment
from gymnasium import Env
# to define action and observation space
from gymnasium.spaces import Discrete, Box


# 0: 'bird'
# 1: 'cactus'
# 2: 'restart'
# 3: 't-rex'

In [2]:
# code to detect the objects in image
from ultralytics import YOLO
import cv2

yolo_model = YOLO("yolo_model/object_identification_model.pt")

def getObjectPositions(observation):
        observation = cv2.cvtColor(observation, cv2.COLOR_GRAY2RGB)
        results = yolo_model.predict(source=observation, conf=0.25, imgsz=640, verbose=False)
        result = results[0]
        boxes = result.boxes

        if len(boxes) == 0:
            return boxes.data

        # Group boxes by class and find leftmost for each class
        class_leftmost = {}

        for i, box in enumerate(boxes):
            class_id = int(box.cls)
            x1 = float(box.xyxy[0][0])  # leftmost x coordinate

            # Keep the box with smallest x1 (leftmost) for each class
            if class_id not in class_leftmost or x1 < class_leftmost[class_id]['x1']:
                class_leftmost[class_id] = {'index': i, 'x1': x1}

        # Extract the indices of leftmost boxes for each class
        leftmost_indices = [info['index'] for info in class_leftmost.values()]

        # Return only the leftmost boxes data
        filtered_data = boxes.data[leftmost_indices]
        return filtered_data


In [3]:
def format_onservation(filtered_data):
    obs = np.array([-1, -1, -1, -1], dtype=np.float32)
    # {0: 'bird', 1: 'cactus', 2: 'restart', 3: 't-rex'}
    #cactus-x1, bird-x1, bird-y1
    for box in filtered_data:
        if int(box[5]) == 1:  # cactus
            obs[0] = box[0]  # x1
            obs[3] = box[0]  # speed
        elif int(box[5]) == 0:  # bird
            obs[1] = box[0]  # x1
            obs[2] = box[1]  # y1
    return obs

In [4]:
# creating the environment
class WebGame(Env):
    def __init__(self):
        super().__init__()
        self.observation_space = Box(
            low=np.array([
                -1,-1,-1, 0  #cactus-x1, bird-x1, bird-y1, speed
            ], dtype=np.float32),
            high=np.array([
                640, 640, 256, 640
            ], dtype=np.float32),
            dtype=np.float32
        )
        self.action_space = Discrete(3)

        # define the extraction parameters for the game
        self.cap = mss()
        monitor_2 = self.cap.monitors[1]
        self.game_location = {
            'top': monitor_2["top"] + 150,
            'left': monitor_2["left"] + 400,
            'width': 640,
            'height': 256
        }

        self.objects_positions = []
        self.pcx = 641
        self.speed = 350
        self.last_speed_calculation_time = -1  # Initialize timestamp

    def calculateSpeed(self, cx):
        if cx == -1:
            return self.speed
        if self.last_speed_calculation_time==-1:
            self.last_speed_calculation_time = time.time()
            self.pcx = cx
            return self.speed
        current_time = time.time()
        time_delta = current_time - self.last_speed_calculation_time
        if time_delta < 0.05:
            return self.speed
        self.last_speed_calculation_time = current_time
        distance = abs(self.pcx - cx)
        currentSpeed = distance / time_delta
        # currentSpeed = max(currentSpeed, 100)
        self.speed = currentSpeed
        self.pcx = cx
        return currentSpeed

    def step(self, action):
        action_map = {
            0: 'space',
            1: 'down',
            2: 'no_op',
        }
        if action != 2:
            pydirectinput.press(action_map[action])

        observation = self.get_observation()
        # Get object positions and names
        try:
            reward, done = self.calculate_reward(self.objects_positions, {0: 'bird', 1: 'cactus', 2: 'restart', 3: 't-rex'})
        except Exception as e:
            print("Error in object detection or reward calculation:", e)
            reward = -10
            done = False

        info = {}
        return observation, reward, done, False, info

    def calculate_reward(self, filtered_data, names):
        """Calculate reward based on T-Rex position relative to obstacles"""
        name_to_id = {v: k for k, v in names.items()}

        trex_box = None
        cactus_box = None
        bird_box = None
        restart_detected = False

        for box in filtered_data:
            class_id = int(box[5])
            if class_id == name_to_id.get('t-rex'):
                trex_box = box
            elif class_id == name_to_id.get('cactus'):
                cactus_box = box
            elif class_id == name_to_id.get('bird'):
                bird_box = box
            elif class_id == name_to_id.get('restart'):
                restart_detected = True

        # Handle collision - end episode immediately
        if restart_detected:
            self.pcx = 641
            self.speed = 350
            self.last_speed_calculation_time = -1
            print('💀', end="")

            # Restart the game automatically
            time.sleep(0.3)
            monitor_2 = self.cap.monitors[1]
            pydirectinput.click(x=monitor_2["left"] + 250, y=monitor_2["top"] + 200)
            pydirectinput.press('space')
            time.sleep(0.3)

            # End episode on any collision
            return -100, True

        if trex_box is None:
            return 0, False

        trex_x1, trex_y1, trex_x2, trex_y2 = [float(x) for x in trex_box[:4]]
        trex_center_x = (trex_x1 + trex_x2) / 2

        # Base reward for survival
        reward = 1.0

        # Check cactus interaction
        if cactus_box is not None:
            cactus_x1, cactus_y1, cactus_x2, cactus_y2 = [float(x) for x in cactus_box[:4]]
            distance = cactus_x1 - trex_x2

            if distance > 50:
                reward += 0.5
            elif 0 < distance < 50:
                if trex_y2 < cactus_y1:
                    print("✅🌵", end=" | ")
                    reward += 5.0
                else:
                    reward -= 2.0

        # Check bird interaction
        if bird_box is not None:
            bird_x1, bird_y1, bird_x2, bird_y2 = [float(x) for x in bird_box[:4]]
            distance = bird_x1 - trex_x2

            if distance > 50:
                reward += 0.5
            elif 0 < distance < 50:
                if trex_y2 > bird_y2 or trex_y1 < bird_y2:
                    print("✅🐦", end=" | ")
                    reward += 5.0
                else  :
                    r  eward -= 2.0

        print(".", end="")
        return reward, False

    def reset(self, seed=None):
        print(f"\n🔄 Episode reset")
        # Reset game state
        time.sleep(0.25)
        monitor_2 = self.cap.monitors[1]
        pydirectinput.click(x=monitor_2["left"] + 250, y=monitor_2["top"] + 200)
        pydirectinput.press('space')
        time.sleep(0.25)
        return self.get_observation(), {}

    def get_observation(self):
        raw = np.array(self.cap.grab(self.game_location))
        raw_rgb = raw[:, :, :3].astype(np.uint8)
        gray = cv2.cvtColor(raw_rgb, cv2.COLOR_RGB2GRAY)
        filtered_data = getObjectPositions(gray)
        self.objects_positions = filtered_data
        formated_observation = format_onservation(filtered_data)
        formated_observation[3] = self.calculateSpeed(formated_observation[0])
        # print(f'[{int(formated_observation[0])},{int(formated_observation[3])}]', end="  ")
        return formated_observation


In [5]:
env = WebGame()
env.get_observation()
# cactus-x1, bird-x1, bird-y1

array([         -1,          -1,          -1,         350], dtype=float32)

In [6]:
for episodes in range(3):
    obs = env.reset(1)
    done = False
    total_reward = 0
    while not done:
        obs, reward, done, truncated, info = env.step(env.action_space.sample())
        total_reward += reward
    print("total reward:", total_reward, ". for episodes:", episodes)


🔄 Episode reset
.............................💀total reward: -69.0 . for episodes: 0

🔄 Episode reset
......

KeyboardInterrupt: 

In [7]:
import os
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import env_checker

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [8]:
env_checker.check_env(env)


🔄 Episode reset

🔄 Episode reset
.
🔄 Episode reset
..........

In [8]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self) -> None:
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True

In [9]:
CHECKPOINT_DIR = './train2/'
LOG_DIR = './logs/'

In [10]:
callback = TrainAndLoggingCallback(check_freq=1000, save_path=CHECKPOINT_DIR)

In [11]:
from stable_baselines3 import DQN

In [13]:
model = DQN('MlpPolicy', env, tensorboard_log=LOG_DIR, verbose=2, buffer_size=10000, learning_starts=1000, learning_rate=0.01)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [14]:
model.learn(total_timesteps=10000000, callback=callback)


🔄 Episode reset
Logging to ./logs/DQN_3
.....................✅🌵 | .....💀
🔄 Episode reset
................💀
🔄 Episode reset
...............✅🌵 | .....💀
🔄 Episode reset
..................💀
🔄 Episode reset
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.2     |
|    ep_rew_mean      | -74.9    |
|    exploration_rate | 1        |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2        |
|    time_elapsed     | 30       |
|    total_timesteps  | 85       |
----------------------------------
.....................💀
🔄 Episode reset
..................💀
🔄 Episode reset
.......................💀
🔄 Episode reset
....................💀
🔄 Episode reset
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.4     |
|    ep_rew_mean      | -77.3    |
|    exploration_rate | 1        |
| time/               |          |
|    episodes         | 8        |
|    fps     

KeyboardInterrupt: 

In [15]:
 # Load the trained model
# model = DQN.load('train2/best_model_70000.zip')
model = DQN.load('yolo_model/best_model_50000.zip')

# Test the model
episodes = 5
for episode in range(episodes):
    obs, _ = env.reset()
    done = False
    total_reward = 0
    step_count = 0

    print(f"Episode {episode + 1} starting...")

    while not done and step_count < 1000:
        # Use the trained model to predict actions
        action, _ = model.predict(obs, deterministic=False)
        print(action, end=" | ")
        action = int(action)  # Convert numpy array to scalar

        obs, reward, done, truncated, info = env.step(action)
        total_reward += reward
        step_count += 1

        # Add small delay to see the gameplay
        time.sleep(0.1)

    print(f"Episode {episode + 1} finished - Total reward: {total_reward}, Steps: {step_count}")



🔄 Episode reset
Episode 1 starting...


ValueError: axes don't match array